## Install the packages

In [ ]:
!pip install -Uqq datasets pythainlp==2.2.4 transformers==4.4.0 tensorflow==2.4.0 tensorflow_text emoji seqeval sentencepiece fuzzywuzzy
!npx degit --force https://github.com/vistec-AI/thai2transformers#dev

In [ ]:
%load_ext autoreload
%autoreload 2

import pythainlp, transformers
pythainlp.__version__, transformers.__version__ #fix pythainlp to stabilize word tokenization for metrics

In [ ]:
import collections
import logging
import pprint
import re
from tqdm.auto import tqdm

import numpy as np
import torch

#datasets
from datasets import (
    load_dataset, 
    load_metric, 
    concatenate_datasets,
    load_from_disk,
)

#transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
)

#thai2transformers
import thai2transformers
from thai2transformers.metrics import (
    squad_newmm_metric,
    question_answering_metrics,
)
from thai2transformers.preprocess import (
    prepare_qa_train_features
)
from thai2transformers.tokenizers import (
    ThaiRobertaTokenizer,
    ThaiWordsNewmmTokenizer,
    ThaiWordsSyllableTokenizer,
    FakeSefrCutTokenizer,
    SEFR_SPLIT_TOKEN
)

from tqdm import tqdm

In [ ]:
model_names = [
    'wangchanberta-base-att-spm-uncased',
    'xlm-roberta-base',
    'bert-base-multilingual-cased',
    'wangchanberta-base-wiki-newmm',
    'wangchanberta-base-wiki-ssg',
    'wangchanberta-base-wiki-sefr',
    'wangchanberta-base-wiki-spm',
]

tokenizers = {
    'wangchanberta-base-att-spm-uncased': AutoTokenizer,
    'xlm-roberta-base': AutoTokenizer,
    'bert-base-multilingual-cased': AutoTokenizer,
    'wangchanberta-base-wiki-newmm': ThaiWordsNewmmTokenizer,
    'wangchanberta-base-wiki-ssg': ThaiWordsSyllableTokenizer,
    'wangchanberta-base-wiki-sefr': FakeSefrCutTokenizer,
    'wangchanberta-base-wiki-spm': ThaiRobertaTokenizer,
}
public_models = ['xlm-roberta-base', 'bert-base-multilingual-cased'] 
#@title Choose Pretrained Model
model_name = "wangchanberta-base-att-spm-uncased" #@param ["wangchanberta-base-att-spm-uncased", "xlm-roberta-base", "bert-base-multilingual-cased", "wangchanberta-base-wiki-newmm", "wangchanberta-base-wiki-syllable", "wangchanberta-base-wiki-sefr", "wangchanberta-base-wiki-spm"]

#create tokenizer
tokenizer = tokenizers[model_name].from_pretrained(
                f'airesearch/{model_name}' if model_name not in public_models else f'{model_name}',
                revision='main',
                model_max_length=416,)

## Loading the model

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained('path/to/mode')

In [ ]:
batch_size = 16
learning_rate = 4e-5

args = TrainingArguments(
    f"finetune_thaiSum",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size*2,
    num_train_epochs=6,
    warmup_ratio=0.15,
    weight_decay=0.01,
    fp16=True,
    save_total_limit=3,
    load_best_model_at_end=True,
)

data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

### Utility functions

In [ ]:
def tokenize_with_space(texts, tokenizer):
  output = []
  encoded_texts = tokenizer(texts, max_length=416, truncation=True)
  for text in encoded_texts['input_ids']:
    tokenized_text = " ".join(tokenizer.convert_ids_to_tokens(text, skip_special_tokens=True))
    if(len(tokenized_text)==0):
      output.append("")
      continue
    if(tokenized_text[0]=="▁"): 
      tokenized_text = tokenized_text[1:]
    output.append(tokenized_text.strip())
  return output

## Install the metrics

### Rouge

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge 
rouge = Rouge()
def cal_rouge_score(hyps, refs, get_average_f1=True):
  '''
  argument: cands, refs [list of string], get_average_f1=True
  return dict of r1, r2, rl score
  if get_average_f1 == True return mean of rouge-1, rouge-2, rouge-L
  '''
  r1 = dict(); r1['precision'] = []; r1['recall'] = []; r1['f1'] = []
  r2 = dict(); r2['precision'] = []; r2['recall'] = []; r2['f1'] = []
  rl = dict(); rl['precision'] = []; rl['recall'] = []; rl['f1'] = []
  for hyp, ref in zip(hyps, refs):
    score = {}
    if(len(hyp)==0 or len(ref)==0):
      score = {
          'rouge-1': {
              'p': 0,
              'r': 0,
              'f': 0
          },
          'rouge-2': {
              'p': 0,
              'r': 0,
              'f': 0
          },
          'rouge-l': {
              'p': 0,
              'r': 0,
              'f': 0
          }
      }
    else: score = rouge.get_scores(hyp, ref)[0]
    r1['precision'].append(score['rouge-1']['p'])
    r1['recall'].append(score['rouge-1']['r'])
    r1['f1'].append(score['rouge-1']['f'])
    
    r2['precision'].append(score['rouge-2']['f'])
    r2['recall'].append(score['rouge-2']['f'])
    r2['f1'].append(score['rouge-2']['f'])

    rl['precision'].append(score['rouge-l']['f'])
    rl['recall'].append(score['rouge-l']['f'])
    rl['f1'].append(score['rouge-l']['f'])
  if(get_average_f1==True): return sum(r1['f1'])/len(r1['f1']), sum(r2['f1'])/len(r2['f1']), sum(rl['f1'])/len(rl['f1'])
  else: return r1, r2, rl

In [ ]:
def evaluate_rouge(cands, refs, tokenizer):
  cands_tokenized = tokenize_with_space(cands, tokenizer)
  refs_tokenized = tokenize_with_space(refs, tokenizer)
  r1, r2, rl = cal_rouge_score(refs_tokenized, cands_tokenized)
  return r1, r2, rl

### BERTScore

In [ ]:
!pip install bert_score==0.3.7

In [ ]:
from bert_score import score
import numpy as np
import gc

In [ ]:
def cal_bert_score(cands, refs, get_average_f1=True):
  '''
  arguments: cands, refs
  return array of presicion, recall, f1, presicion_average, recall_average, f1_average
  if get_average == True return mean of BERTScore
  '''
  p, r, f1 = score(cands, refs, lang="others", verbose=False)
  p = p.numpy()
  r = r.numpy()
  f1 = f1.numpy()
  if(get_average_f1==True): return f1.mean()
  else: return p, r, f1

def cal_batch_bert_score(cands, refs, get_average_f1=True, batch_size=8):
  f1_average = []
  for i in tqdm(range(0,len(cands),batch_size)):
    cand_batch = cands[i:i+batch_size]
    ref_batch = refs[i:i+batch_size]
    res = cal_bert_score(cand_batch, ref_batch)
    f1_average.append(res)
    gc.collect()
  print(f1_average)
  return sum(f1_average)/len(f1_average)

In [ ]:
%%time
refs = ['เมื่อวันที่ 6 ม.ค.60 ที่ทำเนียบรัฐบาล นายวิษณุ เครืองาม รองนายกรัฐมนตรี กล่าวถึงกรณี ที่ นายสุรชัย เลี้ยงบุญเลิศชัย รองประธานสภานิติบัญญัติแห่งชาติ (สนช.) ออกมาระบุว่า การเลือกตั้งจะถูกเลื่อนออกไปถึงปี 2561 ว่า ขอให้ไปสอบถามกับ สนช. แต่เชื่อว่าคงไม่กล้าพูดอีก เพราะทำให้คนเข้าใจผิด ซึ่งที่ สนช.พูดเนื่องจากผูกกับกฎหมายของกรรมการร่างรัฐธรรมนูญ(กรธ.) ตนจึงไม่ขอวิพากษ์วิจารณ์ แต่รัฐบาลยืนยันว่ายังเดินตามโรดแม็ป ซึ่งโรดแม็ปมองได้สองแบบ คือ มีลำดับขั้นตอนและการกำหนดช่วงเวลา โดยเริ่มต้นจากการประกาศใช้รัฐธรรมนูญ แต่ขณะนี้รัฐธรรมนูญยังไม่ประกาศใช้ จึงยังเริ่มนับหนึ่งไม่ถูก จากนั้นเข้าสู่ขั้นตอนการร่างกฎหมายประกอบร่างรัฐธรรมนูญหรือกฎหมายลูก ภายใน 240 วัน ก่อนจะส่งกลับให้ สนช.พิจารณา ภายใน 2 เดือน\xa0,นายวิษณุ กล่าวต่อว่า หากมีการแก้ไขก็จะมีการพิจารณาร่วมกับ กรธ.อีก 1 เดือน ก่อนนำขึ้นทูลเกล้าฯ ทรงลงพระปรมาภิไธย ภายใน 90 วัน และจะเข้าสู่การเลือกตั้งภายในระยะเวลา 5 เดือน ซึ่งทั้งหมดนี้คือโรดแม็ปที่ยังเป็นแบบเดิมอยู่ ส่วนเดิมที่กำหนดวันเลือกตั้งไว้ภายในปี 60 นั้น เพราะมาจากสมมติฐานของขั้นตอนเดิมทั้งหมด แต่เมื่อมีเหตุสวรรคตทุกอย่างจึงต้องเลื่อนออกไป ส่วนการพิจารณากฎหมายลูกทั้งหมด 4 ฉบับ ขณะนี้กรธ.พิจารณาแล้วเสร็จ 2 ฉบับ คือ พ.ร.ป.พรรคการเมือง และพ.ร.ป. คณะกรรมการการเลือกตั้ง แต่ พ.ร.ป.การเลือกตั้งควรจะพิจารณาได้เร็วกลับล่าช้า ดังนั้น กรธ.จะต้องออกชี้แจงถึงเหตุผลว่าทำไมพิจารณากฎหมายดังกล่าวล่าช้ากว่ากำหนด ส่งผลให้เกิดข้อสงสัยจนถึงทุกวันนี้ ส่วนกรณีที่ สนช. ระบุว่า มีกฎหมายเข้าสู่การพิจารณาของ สนช.เป็นจำนวนมาก ทำให้ส่งผลกระทบต่อโรดแม็ปนั้น รัฐบาลเคยบอกไว้แล้วว่าในช่วงนี้ของโรดแม็ปกฎหมายจะเยอะกว่าที่ผ่านมา ดังนั้น สนช.จะต้องบริหารจัดการกันเอง เพราะได้มีการเพิ่มสมาชิก สนช.ให้แล้ว.']
cands = ['เมื่อวันที่ 6 ม.ค.60 ที่ทำเนียบรัฐบาล นายวิษณุ เครืองาม รองนายกรัฐมนตรี กล่าวถึงกรณี ที่ นายสุรชัย เลี้ยงบุญเลิศชัย รองประธานสภานิติบัญญัติแห่งชาติ (สนช.)']
f1_average = cal_bert_score(cands, refs)

In [ ]:
print(f1_average)

## Train and evaluate

Download the data and make it to datasets `dataset`.
The dataset should consist of `attention_mask` and `input_ids`.

In [ ]:
raw_predictions = trainer.predict(dataset)

In [ ]:
predictions = post_process_index(tokenized_datasets['test'], raw_predictions[0], tokenizer)

In [ ]:
r1, r2, rl = evaluate_rouge(predictions, gold_summaries, tokenizer)
print(r1, r2, rl)

In [ ]:
BERTScore = cal_batch_bert_score(predictions, gold_summaries, tokenizer, batch_size=128)
print(BERTScore)